**Ganti Runtime ke GPU sebelum menjalankan kode**

In [ ]:
!nvidia-smi

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

In [ ]:
!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

In [ ]:
from ultralytics import YOLO

from IPython.display import display, Image

In [ ]:
!mkdir {HOME}/datasets
%cd {HOME}/datasets

!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="xxxxxxxxxxxxxxx")
project = rf.workspace("contoh-ndt31").project("go-project")
version = project.version(2)
dataset = version.download("yolov11")

In [ ]:
%cd {HOME}
!yolo task=segment mode=train model=yolo11s-seg.pt data={dataset.location}/data.yaml epochs=100 imgsz=1280 plots=True

In [ ]:
!ls {HOME}/runs/segment/train/

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/segment/train/confusion_matrix.png', width=600)

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/segment/train/results.png', width=600)

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/segment/train/val_batch0_pred.jpg', width=600)

In [ ]:
%cd {HOME}
!yolo task=segment mode=val model={HOME}/runs/segment/train/weights/best.pt data={dataset.location}/data.yaml

In [ ]:
%cd {HOME}
!yolo task=segment mode=predict model={HOME}/runs/segment/train/weights/best.pt conf=0.25 source={dataset.location}/test/images save=True

In [ ]:
import glob
from IPython.display import Image, display

for image_path in glob.glob(f'{HOME}/runs/segment/predict/*.jpg')[:3]:
      display(Image(filename=image_path, width=600))
      print("\n")

In [ ]:
MODEL_PATH = "/content/runs/segment/train/weights/best.pt"
OUT_DIR = "/content/runs/segment/train/weights/"
import os
os.makedirs(OUT_DIR, exist_ok=True)
print("Model:", MODEL_PATH)
print("Out dir:", OUT_DIR)

In [ ]:
from ultralytics import YOLO
import os, glob, shutil

yolo = YOLO(MODEL_PATH)
print("YOLO object loaded:", yolo)

print("Attempting TorchScript export (no fp16)...")
exported = None
try:
    exported = yolo.export(format='torchscript', imgsz=640)
    print("export result:", exported)
except Exception as e:
    print("TorchScript export failed:", e)

candidates = sorted(glob.glob("*.pt"), key=os.path.getmtime, reverse=True)
for p in candidates:
    if "torchscript" in p.lower() or p.endswith(".pt"):
        dst = os.path.join(OUT_DIR, os.path.basename(p))
        try:
            shutil.move(p, dst)
            print("Moved", p, "->", dst)
        except Exception:
            pass

In [ ]:
MODEL_DRIVE_PATH = "/content/runs/segment/train/weights/best.torchscript"
API_SECRET = "xxxxxxxxxxxxxxxxx"
PORT = 5000

In [ ]:
!pip install -q flask flask-cors pyngrok pillow numpy torch torchvision ultralytics

In [ ]:
%%bash
cat > app.py <<'PY'
from flask import Flask, request, jsonify
from flask_cors import CORS
import os, base64, io
from PIL import Image
import torch, numpy as np
from model_utils import load_model, run_inference, save_mask_png_base64

app = Flask(__name__)
CORS(app)

MODEL_PATH = os.environ.get('MODEL_PATH', '/content/runs/segment/train/weights/best.torchscript')
API_SECRET = os.environ.get('API_SECRET', 'xxxxxxxxxxxxx')
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print("Loading model:", MODEL_PATH, "on", DEVICE)
model = load_model(MODEL_PATH, DEVICE)
print("Model loaded.")

@app.route('/health')
def health():
    return jsonify({'status':'ok','device':str(DEVICE)})

@app.route('/infer', methods=['POST'])
def infer():
    auth = request.headers.get('Authorization','')
    if not auth.startswith('Bearer ') or auth.split(' ',1)[1] != API_SECRET:
        return jsonify({'error':'unauthorized'}), 401

    data = request.get_json(force=True, silent=True)
    if not data or 'image' not in data:
        return jsonify({'error':'no image provided'}), 400

    try:
        img_bytes = base64.b64decode(data['image'])
        img = Image.open(io.BytesIO(img_bytes)).convert('RGB')
    except Exception as e:
        return jsonify({'error':'invalid image','detail':str(e)}), 400

    boxes, scores, labels, mask = run_inference(model, img, DEVICE)
    summary = []
    def _as_list(x, idx):
        try:
            if isinstance(x, list):
                return x[idx]
            if _np is not None and isinstance(x, _np.ndarray):
                return x[idx].tolist()
            try:
                import torch as _torch
                if isinstance(x, _torch.Tensor):
                    return x[idx].detach().cpu().numpy().tolist()
            except Exception:
                pass
            try:
                return x[idx].tolist()
            except Exception:
                return x[idx]
        except Exception:
            return None

    n = max(len(boxes) if hasattr(boxes, '__len__') else 0,
            len(scores) if hasattr(scores, '__len__') else 0,
            len(labels) if hasattr(labels, '__len__') else 0)

    for i in range(n):
        box = _as_list(boxes, i)
        score = None
        label = None
        try:
            score = float(scores[i]) if hasattr(scores, '__len__') and i < len(scores) else None
        except Exception:
            try:
                score = float(scores[i].item())
            except Exception:
                score = None
        try:
            label = int(labels[i]) if hasattr(labels, '__len__') and i < len(labels) else None
        except Exception:
            try:
                label = int(labels[i].item())
            except Exception:
                label = None

        if box is None:
            continue
        try:
            if hasattr(box, '__len__') and not isinstance(box, str):
                box_list = [float(x) for x in box]
            else:
                box_list = [float(box)]
        except Exception:
            try:
                box_list = list(map(float, box))
            except Exception:
                box_list = []

        summary.append({'box': box_list, 'score': score, 'label': label})
    mask_b64 = save_mask_png_base64(mask)
    return jsonify({'summary': summary, 'mask_png': mask_b64})

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=int(os.environ.get('PORT', 5000)))
PY

%%bash
cat > model_utils.py <<'PY'
from ultralytics import YOLO
import numpy as np
from PIL import Image
import io, base64, os

def load_model(path, device):
    """
    Load model via Ultralytics YOLO class.
    Works for .pt (best.pt) and TorchScript too (YOLO can wrap it).
    """
    print("Loading model via Ultralytics YOLO from", path)
    yolo = YOLO(path)
    return yolo

def pil_to_np(img):
    """PIL Image -> HWC numpy uint8"""
    return np.array(img)

def run_inference(yolo_obj, pil_img, device):
    """
    Run inference using yolo_obj.predict on a PIL image.
    Returns: boxes (Nx4 numpy), scores (N,), labels (N,), combined_mask (H,W uint8 0/255)
    """
    arr = pil_to_np(pil_img)  # H,W,C
    dev = 'cuda' if str(device).lower().startswith('cuda') else 'cpu'
    results = yolo_obj.predict(source=arr, imgsz=640, conf=0.25, device=dev, verbose=False)
    r = results[0]

    boxes = []
    scores = []
    labels = []
    if hasattr(r, 'boxes') and r.boxes is not None and len(r.boxes) > 0:
        xyxy = r.boxes.xyxy.cpu().numpy()
        conf = r.boxes.conf.cpu().numpy()
        cls = r.boxes.cls.cpu().numpy()
        boxes = [b.tolist() for b in xyxy]
        scores = [float(x) for x in conf]
        labels = [int(x) for x in cls]
    else:
        boxes = []
        scores = []
        labels = []

    combined_mask = None
    if hasattr(r, 'masks') and r.masks is not None:
        try:
            mask_data = r.masks.data.detach().cpu().numpy()  # (N,H,W)
            if mask_data.ndim == 3 and mask_data.shape[0] > 0:
                combined = (mask_data.sum(axis=0) > 0).astype('uint8') * 255
            else:
                combined = (mask_data > 0.5).astype('uint8') * 255
            combined_mask = combined
        except Exception:
            try:
                mask_data = np.array(r.masks.masks)
                combined = (mask_data.sum(axis=0) > 0).astype('uint8') * 255
                combined_mask = combined
            except Exception:
                combined_mask = None
    if combined_mask is None:
        H, W = pil_img.size[1], pil_img.size[0]
        combined_mask = np.zeros((H, W), dtype='uint8')

    return boxes, scores, labels, combined_mask

def save_mask_png_base64(mask_np):
    from PIL import Image
    if mask_np.dtype != 'uint8':
        mask_np = (mask_np * 255).astype('uint8')
    img = Image.fromarray(mask_np)
    buf = io.BytesIO()
    img.save(buf, format='PNG')
    return base64.b64encode(buf.getvalue()).decode('utf-8')
PY

export MODEL_PATH="/content/runs/segment/train/weights/best.torchscript"
export API_SECRET="xxxxxxxxxxxxx"
export PORT="5000"
python -u -c "print('env set')"

In [ ]:
!pip install -q pyngrok requests

from pyngrok import ngrok, conf
import subprocess, time, requests, os

conf.get_default().auth_token = "xxxxxxxxxxxxxxxxxxxxxxxx"

tunnel = ngrok.connect(5000, "http")
public_url = tunnel.public_url
print("ngrok public_url ->", public_url)

os.environ['MODEL_PATH'] = "/content/runs/segment/train/weights/best.torchscript"
os.environ['API_SECRET'] = "xxxxxxxxxxxx"
os.environ['PORT'] = "5000"

cmd = "nohup python app.py > flask_log.txt 2>&1 & echo $!"
try:
    out = subprocess.check_output(cmd, shell=True, text=True).strip()
    print("nohup started, output:", out)
except Exception as e:
    p = subprocess.Popen(["python", "app.py"], stdout=open("flask_log.txt","w"), stderr=open("flask_err.txt","w"))
    print("Started Flask with Popen pid:", p.pid)

time.sleep(6)

print("== flask_log (tail) ==")
!tail -n 40 flask_log.txt || true

try:
    health_resp = requests.get(public_url + "/health", timeout=10)
    print("Health status code:", health_resp.status_code)
    print("Health response:", health_resp.text)
except Exception as e:
    print("Health check failed:", e)
    print("If health failed, inspect flask_log.txt for errors.")


In [ ]:
import base64, requests, os
from IPython.display import display, Image as IPyImage
NGROK = "https://unpensionable-complicative-aiyana.ngrok-free.dev"
API_SECRET = "xxxxxxxxxxxx"

img_path = "/content/sample.jpg"
if not os.path.exists(img_path):
    raise FileNotFoundError(f"File not found: {img_path} - upload sample.jpg ke /content atau ubah path")

with open(img_path, "rb") as f:
    img_b64 = base64.b64encode(f.read()).decode("utf-8")

resp = requests.post(
    NGROK + "/infer",
    headers={"Authorization": f"Bearer {API_SECRET}", "Content-Type": "application/json"},
    json={"image": img_b64},
    timeout=60
)

print("HTTP status:", resp.status_code)
try:
    j = resp.json()
except Exception as e:
    print("Response is not json:", resp.text[:1000])
    raise

print("Response keys:", list(j.keys()))
if "summary" in j:
    print("summary length:", len(j["summary"]))
if "mask_png" in j:
    print("mask_png present:", j["mask_png"] is not None)

mask_b64 = j.get("mask_png", None)
if mask_b64:
    try:
        mask_bytes = base64.b64decode(mask_b64)
        display(IPyImage(data=mask_bytes))
    except Exception as e:
        print("Failed to decode/display mask_png:", e)
else:
    print("No mask_png returned in response.")